In [1]:
import torch
import nip

import os
import sys
sys.path.append("/home/msst/repo/Quantization")
import qlib

DEVICE = 'cuda:0'

In [2]:
#qmodel = torch.load('/home/msst/repo/Quantization/logs/checkpoints_Llama2-7b-hf/kmeans_noscale_percoord.pth')
#qmodel = torch.load('/home/msst/repo/Quantization/logs/checkpoints_Llama2-7b-hf/kmeans_scaleL2_percoord.pth')

qmodel = torch.load('/home/msst/repo/Quantization/logs/checkpoints_Llama2-7b-hf/SymQuant/cb4096_vecdim8_weightPERCOORD_scaleOUTL2_distMSE_blocksizeNone_iters10_abscoords.pth')

/tmp/ipykernel_201750/3003855609.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  qmodel = torch.load('/home/msst/repo/Quantization/logs/checkpoints_Llama2-7b-hf/SymQuant

In [4]:
dataloader = qlib.QATDataset(
    config=nip.load('/home/msst/repo/Quantization/configs/data/wikitext_test_seqlen4096.yaml'),
    tokenizer=qlib.load_tokenizer('Llama2-7b-hf')
).get_dataloader()


with torch.amp.autocast('cuda', dtype=torch.float16):
    res = qlib.evaluate(qmodel.to(DEVICE), dataloader, print_times=25)
    print(res)

  5%|▍         | 4/83 [00:15<05:11,  3.94s/it]

10.247810134281186


  6%|▌         | 5/83 [00:22<05:51,  4.50s/it]


KeyboardInterrupt: 

In [ ]:
path_to_new_blocks = '/home/msst/repo/Quantization/logs/llama2-7b/symquant/llama2_symquant_cb256_vecdim8_C5e-5_L5e-5_lossL1_reassfrac1.0_3layershighbit/per_block_q_trained'
#path_to_new_blocks = '/home/msst/repo/Quantization/logs/llama2-7b/symquant/llama2_symquant_cb256_vecdim8_C1e4_lossL2/per_block_q_trained'
block_names = sorted(os.listdir(path_to_new_blocks), key=lambda x: int(x.split('.')[-1]) if x.startswith("model.layers") else -1)


for block_name in block_names:
    new_block = torch.load(os.path.join(path_to_new_blocks, block_name), weights_only=False)
    curr_block = qmodel.get_submodule(block_name)

    with torch.no_grad():
        curr_block.load_state_dict(new_block.state_dict())



In [6]:
dataloader = qlib.QATDataset(
    config=nip.load('/home/msst/repo/Quantization/configs/data/wikitext_test_seqlen4096.yaml'),
    tokenizer=qlib.load_tokenizer('Llama2-7b-hf')
).get_dataloader()

with torch.amp.autocast('cuda', dtype=torch.float16):
    res = qlib.evaluate(qmodel.to(DEVICE), dataloader, print_times=25)
    print(res)

  5%|▍         | 4/83 [00:15<05:10,  3.93s/it]

6.779945339620922


  8%|▊         | 7/83 [00:27<04:58,  3.93s/it]

6.9054375411395315


 12%|█▏        | 10/83 [00:39<04:47,  3.94s/it]

...

7.176955465175023


 99%|█████████▉| 82/83 [05:19<00:03,  3.91s/it]

7.21580253478738


100%|██████████| 83/83 [05:23<00:00,  3.90s/it]

7.234027145707025


In [5]:
torch.save(qmodel.cpu(), '/home/msst/repo/Quantization/logs/checkpoints_Llama2-7b-hf/SymQuant/cb256_vecdim8_weightPERCOORD_scaleOUTL2_distMSE_blocksizeNone_iters25_abscoords_ptq.pth')

In [18]:
import math

self_attn_matrix_params = 4096 * 4096
mlp_matrix_params = 11008 * 4096
total_params = 4 * self_attn_matrix_params + 3 * mlp_matrix_params

vecdim = 16
cb_size = 2**12

cb_bits = 16 * vecdim * cb_size

q_self_attn_matrix_bits = self_attn_matrix_params // vecdim * (vecdim + math.log2(cb_size)) + cb_bits
q_mlp_matrix_bit = mlp_matrix_params // vecdim * (vecdim + math.log2(cb_size)) + cb_bits

total_bits = 4 * q_self_attn_matrix_bits + 3 * q_mlp_matrix_bit

print(total_bits / total_params)

1.7862694300518134


In [6]:
import safetensors

c = safetensors.torch.load_file('/home/msst/repo/Quantization/notebooks/vptq_checkpoint/model.safetensors')
c

{'lm_head.weight': tensor([[-0.0039,  0.0032, -0.0071,  ...,  0.0053, -0.0082,  0.0070],
         [-0.0315,  0.0466, -0.0023,  ..., -0.0211,  0.0173,  0.0334],
         [-0.0125,  0.0036,  0.0195,  ..., -0.0271,  0.0143, -0.0082],
         ...,
         [-0.0281, -0.0195, -0.0024,  ...,  0.0123, -0.0117, -0.0237],
         [ 0.0229,  0.0255,  0.0315,  ...,  0.0067, -0.0092, -0.0058],
         [ 0.0080, -0.0088,  0.0063,  ..., -0.0293, -0.0200,  0.0337]],
        dtype=torch.float16),
 'model.embed_tokens.weight': tensor([[ 1.2517e-06, -1.7881e-06, -4.3511e-06,  ...,  8.9407e-07,
          -6.5565e-06,  8.9407e-07],
         [ 1.8616e-03, -3.3722e-03,  3.9864e-04,  ..., -8.3008e-03,
           2.5787e-03, -3.9368e-03],
         [ 1.0986e-02,  9.8877e-03, -5.0964e-03,  ...,  2.5177e-03,
           7.7057e-04, -5.0049e-03],
         ...,
         [-1.3977e-02, -2.7313e-03, -1.9897e-02,  ..., -1.0437e-02,
           9.5825e-03, -1.8005e-03],
         [-1.0742e-02,  9.3384e-03,  1.2939e-02,

In [7]:
import torch

c = torch.load('/home/msst/repo/Quantization/notebooks/vptq_checkpoint/model.safetensors')

/tmp/ipykernel_351381/709289361.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  c = torch.load('/home/msst/repo/Quantization/notebooks/vptq_checkpoint/model.safetensors'

UnpicklingError: unpickling stack underflow

In [5]:
import sys
sys.path.append("/home/msst/repo/Quantization")
import qlib
import torch

block = torch.load('/mnt/ssd_storage/ml/weights/vc_data/Llama2-7b-hf/per_block_q/model.layers.0')

/tmp/ipykernel_8534/2624179838.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  block = torch.load('/mnt/ssd_storage/ml/weights/vc_data/Llama2-7b-hf/per_block_q/model.lay

In [6]:
block

LlamaDecoderLayer(
  (self_attn): LlamaAttention(
    (q_proj): HQLinear()
    (k_proj): HQLinear()
    (v_proj): HQLinear()
    (o_proj): HQLinear()
  )
  (mlp): LlamaMLP(
    (gate_proj): HQLinear()
    (up_proj): HQLinear()
    (down_proj): HQLinear()
    (act_fn): SiLU()
  )
  (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
  (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
)

In [5]:
block.self_attn.q_proj

VQLinear()

In [10]:
block.additions = torch.nn.Parameter(torch.randn(10))
print(hasattr(block, 'additions'))

True


In [11]:
del block.additions
print(hasattr(block, 'additions'))

False
